In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np 

Using TensorFlow backend.


In [3]:
import cv2
import numpy as np
import argparse
import zipfile
from tqdm import tqdm
import os
import xmltodict

In [10]:
pp = cv2.imread('Scene_12_shot_0_frame_0_Company_men.png')
pp = np.array(pp,dtype='float32')

In [11]:
type(pp)

numpy.ndarray

In [33]:
np.save('test',pp)

In [35]:
features_path='features_videos/'
p=[]
for video in os.listdir(features_path):
    for shots in os.listdir(features_path+video+'/shots'):
        for image_name in os.listdir(features_path+video+'/shots/'+shots):
            frame = cv2.imread(features_path+video+'/shots/'+shots+'/'+image_name)
            print(image_name,np.shape(frame))
            
            
        print(shots)
        break
        p.append(shots)
    break
#     print(video)
len(p)

2678.png (224, 224, 3)
2677.png (224, 224, 3)
2676.png (224, 224, 3)
(111583, 111623)


0

In [17]:
from pathlib import Path